# Versuch 2 - Kalibrierung von Digitalkameras

Dominik Bartsch, Stefan Ptacek, Niklas Kaiser

In [2]:
#!python -m pip install opencv-python
import cv2
import numpy as np

## 1. Aufnahme und Analyse eines Grauwertkeiles
### 1.1 Einlesen der Werte aus der Kamera

In [2]:
# -------- Aufgabe1.1 -------- #

cap = cv2.VideoCapture(0)

for i in range(10):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("./data/test" + str(i) + ".png", gray)

print("frame width: " + str(cap.get(3)))
print("frame height: " + str(cap.get(4)))
print("--------------------------------")
print("brightness: " + str(cap.get(10)))
print("contrast: " + str(cap.get(11)))
print("saturation: " + str(cap.get(12)))
print("--------------------------------")
print("gain: " + str(cap.get(14)))
print("exposure: " + str(cap.get(15)))
print("--------------------------------")
print("white balance: " + str(cap.get(17)))

cap.release()
#cv2.destroyAllWindows()

frame width: 1920.0
frame height: 1080.0
--------------------------------
brightness: 128.0
contrast: 32.0
saturation: 64.0
--------------------------------
gain: -1.0
exposure: -6.0
--------------------------------
white balance: 1.0


### 1.2 Mittelwert und die Standartabweichung

In [3]:
image = cv2.imread("./data/dark1.png")
image = image.astype('float32')

schwarz = image[0:450,0:130]
dunkelgrau = image[:,140:270]
mittelgrau = image[:,280:410]
hellgrau = image[:,420:540]
weiß = image[:,560:640]

cv2.imwrite("./data/changed/black.png", schwarz.astype('uint8'))
cv2.imwrite("./data/changed/dark1.png", dunkelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark2.png", mittelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark3.png", hellgrau.astype('uint8'))
cv2.imwrite("./data/changed/white.png", weiß.astype('uint8'))

print("\t\t| std \t\t| mean")
print("-----------------------------------------------------")
print("schwarz \t| %f \t| %f" %(np.std(schwarz),np.mean(schwarz)))
print("dunkelgrau \t| %f \t| %f" %(np.std(dunkelgrau),np.mean(dunkelgrau)))
print("mittelgrau \t| %f \t| %f" %(np.std(mittelgrau),np.mean(mittelgrau)))
print("hellgrau \t| %f \t| %f" %(np.std(hellgrau),np.mean(hellgrau)))
print("weiß \t\t| %f \t| %f" %(np.std(weiß),np.mean(weiß)))

		| std 		| mean
-----------------------------------------------------
schwarz 	| 2.641622 	| 29.286991
dunkelgrau 	| 2.657652 	| 68.133286
mittelgrau 	| 1.679518 	| 112.853996
hellgrau 	| 2.370252 	| 151.935501
weiß 		| 2.986076 	| 195.880798


## 2. Aufnahme eines Dunkelbildes

In [4]:
imgpath = "./data/black"
imgpath2 = "./data/white0.png"
imgarr = []


def pixmean(path):
    for i in range(10):
        img = cv2.imread(path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        imgarr.append(img)

    return np.mean(imgarr, axis=0).astype(np.double)

darkmean = pixmean(imgpath)
cv2.imwrite("./data/changed/darkmean.png", darkmean.astype('uint8'))

def submeandark(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    return img - darkmean

new_img = submeandark(imgpath2)
cv2.imwrite("./data/changed/fixedimage.png", new_img.astype('uint8'))

True

### Mittelwert Dunkelbild:
![](data/changed/darkmean.png)

### Ausgangsbild
![](data/changed/fixedimage.png)

## 3. Aufnahme eines Weißbildes

In [6]:
def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img


def bildKontrastMaximiert(img, filename):
    imgCorrected = np.zeros(shape=(len(img), len(img[0])))
    min = np.min(img)
    max = np.max(img)
    for i in range(480):
        for j in range(640):
            imgCorrected[i][j] = img[i][j] - min
            if max > min:
                imgCorrected[i][j] = imgCorrected[i][j] * (255 / (max - min))
            else:
                imgCorrected[i][j] = imgCorrected[i][j] * 255

    cv2.imwrite("data/changed/" + filename + ".png", imgCorrected.astype('float32'))

    count = 0
    for i in range(len(img)):
        for j in range(len(img[0])):
            if img[i][j] == imgCorrected[i][j]:
                count += 1
    return imgCorrected


imagedb = get_image("./data/black") #dunkelbild
# imagedb = imagedb.astype('float32')

imagewb = get_image("./data/white") #weißbild
# imagewb = imagewb.astype('float32')

imagegrauwert = cv2.imread("./data/dark1.png", cv2.IMREAD_GRAYSCALE) # grauwertkeil
imagegrauwert = imagegrauwert.astype('float32')


eingangsbildname = "dark1"
imageub = cv2.imread("./data/" + eingangsbildname + ".png", cv2.IMREAD_GRAYSCALE) #unbearbeitetes einlesebild
imageub = imageub.astype('float32')

darksubtract = np.subtract(imageub,imagedb) # Dunkelbild von Eingangsbild abziehen

cv2.imwrite("./data/changed/black_bearbeitet.png", darksubtract.astype('float32'))

imgCorrected = bildKontrastMaximiert(np.subtract(imagewb, imagedb), "white_kontrastmaximiert")


darkImagesGray = []
for i in range(10):
    image = cv2.imread('./data/dark{}.png'.format(i))
    darkImagesGray.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))


def pixelwiseMeanAndSave(arr, filename):
    imageMean = np.zeros(shape=(len(arr[0]), len(arr[0][0])))

    for i in range(len(arr[0])):
        line = []
        for j in range(len(arr[0][0])):
            values = []
            for k in range(len(arr)):
                values.append(arr[k][i][j])
            line.append(np.mean(values))
        imageMean[i] = line
    #cv2.imwrite('{}.png'.format(filename), imageMean)

    return imageMean


darkImageMean = pixelwiseMeanAndSave(darkImagesGray, "Dunkelbild")

def pixelwiseMeanAndSave(arr, filename):
    imageMean = np.zeros(shape=(len(arr[0]), len(arr[0][0])))

    for i in range(len(arr[0])):
        line = []
        for j in range(len(arr[0][0])):
            values = []
            for k in range(len(arr)):
                values.append(arr[k][i][j])
            line.append(np.mean(values))
        imageMean[i] = line
    #cv2.imwrite('{}.png'.format(filename), imageMean)

    return imageMean

def subtractAndSaveAs(img, darkImg, filename):
    subImg = np.zeros(shape=(len(img), len(img[0])))

    for i in range(len(img)):
        for j in range(len(img[0])):
            subImg[i][j] = img[i][j] - darkImg[i][j]

    #cv2.imwrite('{}.png'.format(filename), subImg)
    return subImg

whiteImagesGray = []
for i in range(10):
    image = cv2.imread('./data/white{}.png'.format(i))
    whiteImagesGray.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

imgWhiteMean = pixelwiseMeanAndSave(whiteImagesGray, "Weissbild")
imgWhiteMean = subtractAndSaveAs(imgWhiteMean, darkImageMean, "Weissbild")

meanwhite = np.divide(imgWhiteMean, np.mean(imgWhiteMean)) # Weißbild normiert mit Mittelwert zu 1
#print(np.mean(imgCorrected))
cv2.imwrite("./data/changed/meanwhitebearbeitet.png", imagewb.astype('float32'))

fin = np.zeros(shape=(len(imagegrauwert), len(imagegrauwert[0])))

for i in range(len(imagegrauwert)):
    for j in range(len(imagegrauwert[0])):
        if meanwhite[i][j] > 0:
            fin[i][j] = float(imagegrauwert[i][j]) / float(meanwhite[i][j])
        else:
            print("Deadpixel x/y: {}/{}".format(j, i))
            fin[i][j] = float(imagegrauwert[i][j])


cv2.imwrite("./data/changed/" + eingangsbildname + "_bearbeitet.png", fin)

True

### Kontrastmaximiertes Weißbild:
![](./data/changed/white_kontrastmaximiert.png)

### Eingangsbild
![](./data/dark1.png)

### Korrigiertes Eingangsbild
![](./dunkelbild.png)

## 4. Pixelfehler

In [24]:
def get_hot_pixels():
    dark_img = get_image("./data/black")
    return np.any(dark_img[:, :] > 0)


def get_stuck_pixels():
    dark_img = get_image("./data/black")
    light_img = get_image("./data/white")
    return np.any(dark_img[:, :] == light_img[:, :])


def get_dead_pixels():
    light_img = get_image("./data/white")
    return np.any(light_img[:, :] == 0)


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


print("Are there Hotpixels? " + ("Yes" if get_hot_pixels() else "No"))
print("Are there Stuckpixels? " + ("Yes" if get_stuck_pixels() else "No"))
print("Are there Deadpixels? " + ("Yes" if get_dead_pixels() else "No"))

Are there Hotpixels? No
Are there Stuckpixels? No
Are there Deadpixels? No


In [8]:
image = cv2.imread("./dunkelbild.png").astype('float32')

schwarz = image[0:450,0:130]
dunkelgrau = image[:,140:270]
mittelgrau = image[:,280:410]
hellgrau = image[:,420:540]
weiß = image[:,560:640]

cv2.imwrite("./data/changed/4black.png", schwarz.astype('uint8'))
cv2.imwrite("./data/changed/4dark1.png", dunkelgrau.astype('uint8'))
cv2.imwrite("./data/changed/4dark2.png", mittelgrau.astype('uint8'))
cv2.imwrite("./data/changed/4dark3.png", hellgrau.astype('uint8'))
cv2.imwrite("./data/changed/4white.png", weiß.astype('uint8'))

print("\t\t| std \t\t| mean")
print("-----------------------------------------------------")
print("schwarz \t| %f \t| %f" %(np.std(schwarz),np.mean(schwarz)))
print("dunkelgrau \t| %f \t| %f" %(np.std(dunkelgrau),np.mean(dunkelgrau)))
print("mittelgrau \t| %f \t| %f" %(np.std(mittelgrau),np.mean(mittelgrau)))
print("hellgrau \t| %f \t| %f" %(np.std(hellgrau),np.mean(hellgrau)))
print("weiß \t\t| %f \t| %f" %(np.std(weiß),np.mean(weiß)))

		| std 		| mean
-----------------------------------------------------
schwarz 	| 2.133742 	| 29.205402
dunkelgrau 	| 2.261158 	| 68.050980
mittelgrau 	| 1.468097 	| 112.709122
hellgrau 	| 2.043680 	| 151.782059
weiß 		| 2.629327 	| 195.660965


### Korrigierter Grauwertkeil
![](./dunkelbild.png)

Todo:
- Kontrastmaximierung bei Weißbild
- Weißbildkorrektur